In [1]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [3]:
print(f"Misssing Values: {data.isnull().sum()}")

Misssing Values: age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [4]:
data['age'] = pd.cut(data['age'], bins = 3, labels =['Young', 'Middle', 'Old'])
data['chol'] = pd.cut(data['chol'], bins = 3, labels = ['Low', 'Normal', 'High'])
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,Middle,1,0,125,Low,0,1,168,0,1.0,2,2,3,0
1,Middle,1,0,140,Low,1,0,155,1,3.1,0,0,3,0
2,Old,1,0,145,Low,0,1,125,1,2.6,0,0,3,0
3,Middle,1,0,148,Low,0,1,161,0,0.0,2,1,3,0
4,Old,0,0,138,Normal,1,1,106,0,1.9,1,3,2,0


In [5]:
model = BayesianNetwork([('age', 'target'),
                        ('chol', 'target'),
                        ('cp', 'target'),
                        ('target', 'thalach')])
model.edges()

OutEdgeView([('age', 'target'), ('target', 'thalach'), ('chol', 'target'), ('cp', 'target')])

In [6]:
model.fit(data, estimator = MaximumLikelihoodEstimator)

In [7]:
infer = VariableElimination(model)
result = infer.query(variables=['target'], evidence={'age': 'Old', 'chol': 'High'})
print(result)

+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        0.3615 |
+-----------+---------------+
| target(1) |        0.6385 |
+-----------+---------------+


In [8]:
result = infer.query(variables = ['target'], evidence={'age': 'Middle', 'chol': 'Normal', 'cp': 2})
print(result)

+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        0.0000 |
+-----------+---------------+
| target(1) |        1.0000 |
+-----------+---------------+
